# Notebook to explore the BOLDMoments data

**Data location:** /oliva/blahner/BOLDMoments/preprocessed_data/responses.

Reminder that reconstruction should be done per subject. Subject 1 is good. 
To get started, use: ./sub01/testing/X_TRavg-56789testing.pkl, where X is either the ROI name or WB (whole brain). The contents of the pkl file is a dict with keys "test_data" and "train_data" with shapes (num_videos, num_repetitions, num_voxels). Feel free to average over repetitions. You can also find the metadata json file and video stimuli in /oliva/datasets/BOLDMoments/metadata.


For time-resolved brain activity, use /sub01/testing/X_TRavg-Ytesting.pkl where Y is 5, 6, 7, 8, or 9. Don't worry about numbers 1-4. But I'd first start with the 56789avg to make sure we can get some reconstruction.

In [1]:
import sys
import os
import pickle as pkl

In [ ]:
# Load pkl file with moments data

with open("./sub01/testing/X_TRavg-56789testing.pkl") as f:
    pkl.load(f)